# Flight Delay prediction

In [1]:
import numpy as np
import pandas as pd
import torch
import datetime
from torch.utils import data
from random import sample,seed

In [2]:
a = np.zeros((15,),dtype =int)
a[2] =1

In [3]:
a

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Importing Flight Data

In [4]:
flights = pd.read_csv("./data/flights.csv")
airlines = pd.read_csv("./data/airlines.csv")
airports = pd.read_csv("./data/airports.csv")

/home/sachin/anaconda3/envs/deep/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(len(airlines))
print(len(airports))

14
322


In [7]:
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [8]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [9]:
airports = {ch:i for i,ch in enumerate(flights.ORIGIN_AIRPORT.unique())}
airlines = {ch:i for i,ch in enumerate(airlines.IATA_CODE)}
airports['10666'] =len(airports)

In [10]:
len(airports)

931

In [11]:
len(airlines)

14

In [12]:
# one hot encoder for day of the week
def one_hot_encode(size,val):
    a = np.zeros((size,),dtype=int)
    a[(val-1)] = 1
    return a

In [13]:
# Date and time preprocess copied from
# link - https://www.kaggle.com/fabiendaniel/predicting-flight-delays-tutorial/data
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

In [14]:
# flights.dropna(inplace = True)
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])

In [15]:
flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_heure) #create_flight_time(flights, 'SCHEDULED_DEPARTURE')
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_heure)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_heure)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_heure)
#__________________________________________________________________________
flights.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
             'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY
0,00:05:00,04:30:00,23:54:00,04:08:00,-11.0,-22.0
1,00:10:00,07:50:00,00:02:00,07:41:00,-8.0,-9.0
2,00:20:00,08:06:00,00:18:00,08:11:00,-2.0,5.0
3,00:20:00,08:05:00,00:15:00,07:56:00,-5.0,-9.0
4,00:25:00,03:20:00,00:24:00,02:59:00,-1.0,-21.0
5,00:25:00,06:02:00,00:20:00,06:10:00,-5.0,8.0


In [16]:
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'DAY','DATE', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME']
flights.drop(variables_to_remove, axis = 1, inplace = True)
flights = flights[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
        'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'SCHEDULED_TIME', 'ELAPSED_TIME','MONTH','DAY_OF_WEEK']]
flights[:5]

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,SCHEDULED_TIME,ELAPSED_TIME,MONTH,DAY_OF_WEEK
0,AS,ANC,SEA,00:05:00,23:54:00,-11.0,04:30:00,04:08:00,-22.0,205.0,194.0,1,4
1,AA,LAX,PBI,00:10:00,00:02:00,-8.0,07:50:00,07:41:00,-9.0,280.0,279.0,1,4
2,US,SFO,CLT,00:20:00,00:18:00,-2.0,08:06:00,08:11:00,5.0,286.0,293.0,1,4
3,AA,LAX,MIA,00:20:00,00:15:00,-5.0,08:05:00,07:56:00,-9.0,285.0,281.0,1,4
4,AS,SEA,ANC,00:25:00,00:24:00,-1.0,03:20:00,02:59:00,-21.0,235.0,215.0,1,4


In [17]:
# flights.dropna(inplace = True)
flights['AIRLINE'] = flights['AIRLINE'].apply(lambda x: airlines[x])
flights['ORGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].apply(lambda x: airports[x])
flights['DESTINATION__AIRPORT'] = flights['DESTINATION_AIRPORT'].apply(lambda x: airports[x])
flights['DAY_OF_WEEK'] = flights['DAY_OF_WEEK'].apply(lambda x: one_hot_encode(7,x))
flights = flights[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT','ARRIVAL_DELAY','DAY_OF_WEEK','DEPARTURE_DELAY']]

In [18]:
flights[:5]

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ARRIVAL_DELAY,DAY_OF_WEEK,DEPARTURE_DELAY
0,6,ANC,SEA,-22.0,"[0, 0, 0, 1, 0, 0, 0]",-11.0
1,1,LAX,PBI,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-8.0
2,2,SFO,CLT,5.0,"[0, 0, 0, 1, 0, 0, 0]",-2.0
3,1,LAX,MIA,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-5.0
4,6,SEA,ANC,-21.0,"[0, 0, 0, 1, 0, 0, 0]",-1.0


In [19]:
flights.dropna(inplace =True)
flights = flights.reset_index()
start = 0
data_batch = {}
labels = {}
for i,x in enumerate(range(0,len(flights),200)):
#     print('id_{}:{}'.format(i,x))
    data_batch['id_{}'.format(i)] = [i for i in range(start,x)]
    labels['id_{}'.format(i)] = flights.loc[x,'DEPARTURE_DELAY']
    start = x

In [25]:
class DataGen(data.Dataset):
    def __init__(self,list_id,labels):
        self.list_id = list_id
        self.labels = labels
    def __len__(self):
        return len(self.list_id)
    def __getitem__(self,index):
        ID = self.list_id[index]
        ind = data_batch[ID]
        X = flights.loc[ind,:'DAY_OF_WEEK']
        y = labels[ID]
        return torch.tensor(X.DAY_OF_WEEK.to_list()),y

In [26]:
params = {'batch_size': 3,
          'shuffle': False,
          'num_workers': 6}

In [27]:
seed(1234)
length = int(len(data_batch.keys())*0.75)
training_ids = sample(list(data_batch.keys()),length)
validation_ids = []
for i in list(data_batch.keys()):
    if i not in training_ids:
        validation_ids.append(i)

In [28]:
training_set = DataGen(training_ids, labels)
training_generator = data.DataLoader(training_set, **params)
validation_set = DataGen(validation_ids, labels)
validation_generator = data.DataLoader(validation_set, **params)

In [29]:
x ,y = next(iter(training_generator))# training_set.__getitem__(0)

In [30]:
x

tensor([[[0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 1, 0]],

        [[0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]]])